# Electoral district

In [580]:
import json
import pandas as pd
from faker import Faker

In [581]:
post_code_df = pd.read_csv('_62_postcode.csv').drop_duplicates(subset=['จังหวัด','อำเภอ'])
electoral_district_df = pd.read_csv('66_Vote62_election-zones.csv')

In [582]:
electoral_district_df[electoral_district_df.duplicated(['จังหวัด', 'อำเภอ', 'ตำบล'])]

,จังหวัด,อำเภอ,ตำบล,เขตเลือกตั้ง,index,เทศบาล,เขตเลือกตั้ง.1
1992,นครพนม,เมืองนครพนม,หนองแสง,2,นครพนมเมืองนครพนมหนองแสง,เทศบาลเมืองนครพนม,2
1993,นครพนม,เมืองนครพนม,อาจสามารถ,2,นครพนมเมืองนครพนมอาจสามารถ,เทศบาลเมืองนครพนม,2
2364,นครศรีธรรมราช,ท่าศาลา,ท่าศาลา,9,นครศรีธรรมราชท่าศาลาท่าศาลา,เทศบาลตำบลท่าศาลา,9
2435,นครศรีธรรมราช,เมืองนครศรีธรรมราช,ท่าไร่,1,นครศรีธรรมราชเมืองนครศรีธรรมราชท่าไร่,เทศบาลตำบลปากนคร,1
2441,นครศรีธรรมราช,เมืองนครศรีธรรมราช,บางจาก,1,นครศรีธรรมราชเมืองนครศรีธรรมราชบางจาก,เทศบาลตำบลบางจาก,1
2443,นครศรีธรรมราช,เมืองนครศรีธรรมราช,ปากนคร,1,นครศรีธรรมราชเมืองนครศรีธรรมราชปากนคร,เทศบาลตำบลปากนคร,1
2445,นครศรีธรรมราช,เมืองนครศรีธรรมราช,ปากพูน,2,นครศรีธรรมราชเมืองนครศรีธรรมราชปากพูน,เทศบาลตำบลท่าแพ,2
2448,นครศรีธรรมราช,เมืองนครศรีธรรมราช,นาเคียน,2,นครศรีธรรมราชเมืองนครศรีธรรมราชนาเคียน,NaN,2
2449,นครศรีธรรมราช,เมืองนครศรีธรรมราช,โพธิ์เสด็จ,1,นครศรีธรรมราชเมืองนครศรีธรรมราชโพธิ์เสด็จ,เทศบาลนครนครศรีธรรมราช,1
2450,นครศรีธรรมราช,เมืองนครศรีธรรมราช,นาเคียน,1,นครศรีธรรมราชเมืองนครศรีธรรมราชนาเคียน,เทศบาลนครนครศรีธรรมราช,1


district object
```
{'district': 'คลองท่อม',
 'province': 'กระบี่',
 'subDistrict': [{name: 'ตำบล 1'}, ...,],
 'electoral': [{'fk': 'กระบี่-3'}]}
```

In [583]:
electoral_district_df = electoral_district_df.assign(
    pk=electoral_district_df.apply(lambda row: "{}-{}".format(row['จังหวัด'], row['เขตเลือกตั้ง.1']), axis=1))

In [584]:
district_list = list()
for (province,district,subdistrict,thesaban), gdf in (electoral_district_df
                                                      .fillna('')
                                                      .groupby(['จังหวัด', 'อำเภอ', 'ตำบล', 'เทศบาล'])):
    district_list.append(dict(
        province=province,
        district=district,
        subDistrict=subdistrict,
        thesaban=thesaban,
        electoralFk=gdf['pk'].tolist(),
    ))

In [585]:
belongs_to_one_electoral_district = (electoral_district_df.groupby(['จังหวัด', 'อำเภอ',])['pk'].apply(
    lambda elect_dist: len(elect_dist.unique()) == 1
))

In [586]:
belongs_to_one_electoral_district.loc[('กระบี่', 'อ่าวลึก')]

True

In [587]:
electdistrict = dict()
for (province, elec_num, elecPk), gdf in electoral_district_df.groupby(['จังหวัด', 'เขตเลือกตั้ง.1', 'pk']):
    _district_list=[]
    for (district), subdist_df in gdf.groupby(['อำเภอ']):
        sub_districts = []
        for subd, thesaban in subdist_df[['ตำบล','เทศบาล']].values:
            if pd.isna(thesaban):
                sub_districts.append(subd)
            else:
                sub_districts.append(f'{subd} ({thesaban})')

        _district_list.append(dict(
            name=district,
            subDistricts=sub_districts,
            belongsToOneElecDist=bool(belongs_to_one_electoral_district.loc[(province,district)]))
        )
    electdistrict[elecPk] = dict(
        province=gdf['จังหวัด'].tolist()[0],
        districts=_district_list,
        electoralDistrictNumber=int(elec_num),
    )

In [588]:
with open('district_province_list.json', 'w') as fp:
    json.dump(district_list, fp, ensure_ascii=False)

In [589]:
with open('electoral_district_table.json', 'w') as fp:
    json.dump(electdistrict, fp, ensure_ascii=False)

# Candidates data

In [590]:
import re
import requests
import os
import numpy as np

In [591]:
electoral_dir = 'electorals'
os.makedirs(electoral_dir, exist_ok=True)

In [592]:
candidates_df = pd.read_csv('66_WV_Candidates.csv')
party_number_df = pd.read_csv('66_WV_Party.csv')
pm_candidates_df = pd.read_csv('66_WV_PMCandidate.csv')
twfu_parties=pd.read_csv('./Parties_exported_1.csv').set_index('Id',)
twfu_people=pd.read_csv('./People_exported_1.csv')

In [593]:
candidates_df.loc[candidates_df.zone.isna(), 'zone'] = -1
# change data type
candidates_df.loc[:,'zone'] = candidates_df.zone.astype(int)

In [594]:
# set party with no number to -1
party_number_df.loc[:, 'partylist_no'] = party_number_df.partylist_no.apply(
    lambda x: int(x) if str(x).isdigit() else -1)

In [595]:
# filter
twfu_parties = twfu_parties[twfu_parties.PartyType == 'พรรค']
# get url
def get_url(s):
    if isinstance(s, str):
        url = re.sub('.*\((.*)\)','\\1', s)
        if url != 'undefined': return url
    return ''
twfu_parties.loc[:,'image_url'] = twfu_parties.Images.apply(get_url)

In [596]:
# create table
party_data = twfu_parties[['image_url', 'Name']].set_index('Name').to_dict()

In [597]:
# add number
party_data['number'] = party_number_df.set_index('party').to_dict()['partylist_no']

In [598]:
# add website
party_data['website'] = twfu_parties[~twfu_parties['Website'].isna()][
    ['Website', 'Name']].set_index('Name').to_dict()

In [599]:
people_image_url = twfu_people.set_index('Name').Images.apply(get_url).to_dict()

In [600]:
# add Prime minister candidates
party_data['pm_candidates'] = dict()
for party, party_df in pm_candidates_df.sort_values('order').groupby('party'):
    party_data['pm_candidates'][party] = [{'Name': name,
                                           'Image': people_image_url.get(name,'')}
                                          for name in party_df.name]

In [601]:
# promisetracker party
resp = requests.get('https://raw.githubusercontent.com/wevisdemo/promise-tracker/main/data/parties.json')
promisetracker_parties = resp.json()

In [602]:
party_data['promisetracker_urls'] = dict()
for party in promisetracker_parties:
    party_name_mabe_many = party['name']
    for party_name in party_name_mabe_many.split('/'): # อนาคตใหม่/ก้าวไกล
        party_data['promisetracker_urls'][party_name] =\
            'https://promisetracker.wevis.info/explore?party='+party_name_mabe_many

In [603]:
# law watch
resp = requests.get('https://raw.githubusercontent.com/wevisdemo/law-watch/main/src/data/parties.ts')
law_watch_parties_string = re.findall('\[[^\]]*\]', resp.content.decode().replace('\n',' '))

In [604]:
party_data['law_watch_urls'] = dict()
for list_string in law_watch_parties_string[:2]:
    party_list = json.loads(re.sub('\'', '"',list_string))
    for party in party_list:
        party_data['law_watch_urls'][party] = 'https://wevis.info/law-watch#investigate-section'

In [605]:
# They work for us
# if the party has ever been a government or opposition
theyworkforus_parties = twfu_parties[~twfu_parties.PartyGroup.isna()].Name.tolist()
party_data['theyworkforus_urls'] = {party: f'https://theyworkforus.wevis.info/party/{party}'
                                for party in theyworkforus_parties}

In [606]:
parties_table_ep = (
    'https://sheets.wevis.info/api/v1/db/public/shared-view/'
    '40065196-c978-4d7a-b3fb-fb84694383a7'
    '/rows'
    '?'
    'l=900'
    '&f=Name,PartyType,PartyGroup,Images,IsActive,Website'
)
resp = requests.get(parties_table_ep)
parties_table = resp.json()['data']['list']

In [607]:
def get_image_url(image):
    if 'url' in image.keys():
        return image['url']
    elif 'path' in image.keys():
        return 'https://sheets.wevis.info/' + image['path'] 

In [608]:
# image url
for party in parties_table:
    if isinstance(party['Images'], list) and len(party['Images']): # if has image
        image = party['Images'][0]
        party_data['image_url'][party['Name']] = get_image_url(image)

In [609]:
people_party_history_ep = (
    'https://sheets.wevis.info/api/v1/db/public/shared-view/'
    '572c5e5c-a3d8-440f-9a70-3c4c773543ec'
    '/rows'
    '?'
    'nested[PeoplePartyHistory][fields]=Party,EstablishedDate'
    '&'
    'f=Name,PeoplePartyHistory,Id,Images'
    '&'
    'l=900')
resp = requests.get(people_party_history_ep)
people_history_table = resp.json()['data']['list']

In [610]:
DISSOVED_DATE = '2023-03-01'

In [611]:
person_is_government = dict()
person_is_opposition = dict()
for person in people_history_table:
    if isinstance(person['Images'], list) and len(person['Images']): # if has image
        image = person['Images'][0]
        people_image_url[person['Name']] = get_image_url(image)
    for party in person['PeoplePartyHistory']:
        if party['EstablishedDate'] is not None and (party['EstablishedDate'] > DISSOVED_DATE):
            person['PeoplePartyHistory'].remove(party)
        else:
            party_id = party['Party']['Id']
            if party_id in twfu_parties.index:
                party_group = twfu_parties.loc[party_id].PartyGroup
                person_is_government[person['Name']] =\
                    person_is_government.get(person['Name'], False) or (party_group == 'ร่วมรัฐบาล')
                person_is_opposition[person['Name']] =\
                    person_is_opposition.get(person['Name'], False) or (party_group == 'ฝ่ายค้าน')

In [612]:
party_data['is_government'] = (twfu_parties[~twfu_parties['PartyGroup'].isna()]\
                                .set_index('Name')['PartyGroup'] == 'ร่วมรัฐบาล')\
                                .to_dict()
party_data['is_opposition'] = (twfu_parties[~twfu_parties['PartyGroup'].isna()]
                               .set_index('Name')['PartyGroup'] == 'ฝ่ายค้าน')\
                                .to_dict()

In [613]:
twfu_people.set_index('Name')[['Birthdate','Education','ExOccupation0']].loc['ประยุทธ์ จันทร์โอชา','Birthdate']

'1954-03-21'

In [614]:
person_table_name_as_index = twfu_people.set_index('Name')

In [615]:
def data_getter(key, person_name):
    try:
        return person_table_name_as_index[~person_table_name_as_index[key].isna()].loc[person_name, key]
    except KeyError:
        return None

In [616]:
people_list = []
party_table = dict()
for i, row in candidates_df.iterrows():
    if row.party not in party_table.keys():
        party_table[row.party] = dict(
            Name=row.party,
            Number=party_data['number'][row.party],
            Image=party_data['image_url'].get(row.party),
            Candidate=party_data['pm_candidates'].get(row.party, []),
            PastGovernment=party_data['is_government'].get(row.party, False),
            PastOpposition=party_data['is_opposition'].get(row.party, False),
            # TODO: fix this on production
            Policy=f'https://election66.wevis.info/policyshop/',
            Promise=party_data['promisetracker_urls'].get(row.party),
            Law=party_data['law_watch_urls'].get(row.party),
            Others=party_data['theyworkforus_urls'].get(row.party),
            Website=party_data['website'].get(row.party),
            PartyList=list()
        )
    is_past_mp = person_is_government.get(row['name'], False) or person_is_opposition.get(row['name'], False)
    if row.mptype == 'เขต':
        people_list.append(dict(
            Name=row['name'],
            Number=row['number'],
            Birthdate=data_getter('Birthdate', row['name']), # TODO:
            Education=data_getter('Education', row['name']), # TODO:
            ExOccupation=data_getter('ExOccupation0', row['name']), # TODO
            Party=row.party,
            Province=row.province,
            Zone=row.zone,
            Image=people_image_url.get(row['name']),
            PastMP=is_past_mp,
            PastGovernment=person_is_government.get(row['name'], False),
            PastOpposition=person_is_opposition.get(row['name'], False),
        ))
    elif row.mptype == 'บัญชีรายชื่อ':
        party_table[row.party]['PartyList'].append(dict(
            Name=row['name'],
            Number=row.number,
            PastMP=is_past_mp,
            # PastGovernment=person_is_government.get(row['name'], False),
            # PastOpposition=person_is_opposition.get(row['name'], False),
        ))

In [619]:
for dist_key, district in electdistrict.items():
    electoral_people_list = [p for p in people_list
                             if p['Province'] == district['province'] # filter out
                             and p['Zone'] == district['electoralDistrictNumber']]
    
#     electoral_parties = [p['Party'] for p in electoral_people_list if p['Number'] != -1]
#     electoral_parties.sort(key=lambda p: party_table[p]['Number'])
    file_path = os.path.join(electoral_dir,f'{dist_key}.json')
    electoral_data = dict(People=sorted(electoral_people_list,key=lambda p: p['Number']),)
    
    with open(file_path,'w') as fp:
        json.dump(electoral_data, fp, ensure_ascii=False)

In [618]:
with open('parties.json', 'w') as fp:
    json.dump(party_table, fp, ensure_ascii=False)